In [ ]:
import trimesh as tm
mesh_file = '/home/motion/Optimized-UV-Disinfection/data/environment_meshes/Single Scene Example - ScanNet0000_00/estimate_native_5_cm.ply'

# Using Trimesh

In [ ]:
from alphashape import alphashape
import shapely
import numpy as np

In [ ]:
a = tm.exchange.load.load(mesh_file)
a.show()

In [ ]:
conn_components = tm.graph.split(a, only_watertight = False)

import random
# maximum_connected_component = 
max_faces = 0
random.shuffle(conn_components)
for component in conn_components:
    if(component.faces.shape[0]>max_faces):
        max_faces = component.faces.shape[0]
        maximum_connected_component  = component
maximum_connected_component.show()

In [ ]:
%%time 
projection = maximum_connected_component.vertices[:,:2]
alpha_shape = alphashape(projection,2)
alpha_shape

In [ ]:
dir(alpha_shape)

In [ ]:
%%time
bounds = alpha_shape.bounds
resolution = 0.25
robot_height = 1.5
x_ = np.arange(bounds[0] - resolution,bounds[2]+resolution,resolution)
y_ = np.arange(bounds[1] - resolution,bounds[3]+resolution,resolution)
z_ = np.arange(0,robot_height+resolution,resolution)
x,y,z = np.meshgrid(x_,y_,z_,indexing = 'ij')
x = x.flatten()
y = y.flatten()
z = z.flatten()
sampling_places = np.zeros(shape = (z.shape[0],3))
sampling_places[:,0] = x
sampling_places[:,1] = y
sampling_places[:,2] = z
multipoint = shapely.geometry.MultiPoint(sampling_places[:,:2])
result = []
for point in list(multipoint):
    result.append(alpha_shape.contains(point))
sampling_places = sampling_places[result]
# result = alpha_shape.intersection(multipoint)

In [ ]:
sampling_places.shape

In [ ]:
np.logical_and(sampling_places[:,0] in result[:,0],sampling_places[:,1] in result[:,1])

In [ ]:
def get_maximum_connected_component_mesh(mesh_file):
    full_mesh = tm.exchange.load.load(mesh_file)
    conn_components = tm.graph.split(full_mesh, only_watertight = False)
    max_faces = 0
    for component in conn_components:
        if(component.faces.shape[0]>max_faces):
            max_faces = component.faces.shape[0]
            maximum_connected_component  = component
    return maximum_connected_component

def get_sampling_places(mesh_file,resolution = 5000,robot_height = 1.5,convex_scale = 0.9,alpha = 2):
    resolution = resolution/1000
    mesh = get_maximum_connected_component_mesh(mesh_file)
    projection = mesh.vertices[mesh.vertices[:,2]<0.3,:2]

    alpha_shape = alphashape(projection,alpha)
    max_area = 0
#     if(len(alpha_shape) > 1):
#         for shape in alpha_shape:
#             if(shape.area>max_area):
#                 max_area = shape.area
#                 final_shape = shape
#         alpha_shape = final_shape
    bounds = alpha_shape.bounds
    x_ = np.arange(bounds[0] - resolution,bounds[2]+resolution,resolution)
    y_ = np.arange(bounds[1] - resolution,bounds[3]+resolution,resolution)
    z_ = np.arange(0,robot_height+resolution,resolution)
    x,y,z = np.meshgrid(x_,y_,z_,indexing = 'ij')
    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    sampling_places = np.zeros(shape = (z.shape[0],3))
    sampling_places[:,0] = x
    sampling_places[:,1] = y
    sampling_places[:,2] = z
    print(sampling_places.shape)
    multipoint = shapely.geometry.MultiPoint(sampling_places[:,:2])
    result = []
    for point in list(multipoint):
        result.append(alpha_shape.contains(point))
    sampling_places = sampling_places[result]
    print("\n\n\n\n this is the shape of sampling places  = {} \n\n\n\n\n".format(sampling_places.shape))
    
    return sampling_places,mesh,alpha_shape,multipoint

a, b, alpha_shape,mp = get_sampling_places(mesh_file,resolution = 250)

In [ ]:
alpha_shape.buffer()

In [ ]:
type(alpha_shape.buffer(-0.5))

In [ ]:
multipoint

In [ ]:
b.show()

In [ ]:
a.visual.face_colors.shape

In [ ]:
a.faces.shape

In [ ]:
from visibility import Visibility

mesh_file2 = './data/environment_meshes/remeshed_hospital_room_full_35k.obj'
vis_tester =Visibility(mesh_file2,res = 512, useShader = True,createWnd = True)
m = vis_tester.mesh

In [ ]:
m.mesh.visual.face_colors

In [ ]:
from glob import glob
import pandas as pd
import trimesh as tm
a = glob('/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/*.ply')

b = pd.Series(a)
c = ~b.str.split('/',expand = True).iloc[:,-1].str.startswith('gt')

estimates = b[c]

d = sorted(estimates.tolist())

# e = tm.exchange.load.load(d[0])
# e.show()
a, b, alpha_shape,mp = get_sampling_places(d[5],resolution = 500,alpha = 10)
alpha_shape

In [ ]:
type(alpha_shape) == shapely.geometry.polygon.Polygon

In [ ]:
b.show()

In [ ]:
type(alpha_shape)

In [ ]:
from glob import glob
experiment = 'soft_semantic_thresholding' #'surface_agnostic'
import pandas as pd
done_meshes = pd.Series(glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/{}/armbot/'.format(experiment))).str.split('/',expand = True).iloc[:,-4].tolist()
done_meshes

# Evaluation Code

In [ ]:
import pandas as pd
from visibility import Visibility
from glob import glob
import trimesh as tm
import pickle
import os
import numpy as np
from scipy.sparse import lil_matrix
from tqdm import tqdm

def get_irradiance_matrix(vis_tester,sampling_places):
    total_faces = vis_tester.mesh.mesh.faces.shape[0]
    irradiance_matrix = lil_matrix((sampling_places.shape[0],total_faces))
    for i in range(sampling_places.shape[0]):
        _,irradiance = vis_tester.render(id0 =None,id1 = None,pos = sampling_places[i,:].tolist())
        irradiance = irradiance
        irradiance_matrix[i,np.where(irradiance > 0)] = irradiance[irradiance > 0]

    return irradiance_matrix.tocsr()

def evaluate_semantic_disinfection_performance(roadmap):
    mesh_name = roadmap.split('/')[-4]
    # we then load the mesh original mesh and its ground truth:
    mesh_file = '/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/{}.ply'.format(mesh_name)
    mesh_gt_file = '/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/gt_{}.ply'.format(mesh_name)
    ground_truth_visibility = Visibility(mesh_gt_file,res = 512, useShader = True,createWnd = True)
    main_dir = os.path.dirname(roadmap)
    reachable_file = os.path.join(main_dir,'armbot_reachable_330_divs.p')
    solutions_file = os.path.join(main_dir,'armbot_solutions_330_divs.p')
    sampling_places_file = os.path.join(main_dir,'armbot_sampling_places_330_divs.p')

    reachable = pickle.load(open(reachable_file,'rb'))
    solutions = np.array(pickle.load(open(solutions_file,'rb'))[0])
    sampling_places = pickle.load(open(sampling_places_file,'rb'))

    selected_points = sampling_places[reachable,:][solutions>0,:]
    irradiance_matrix = get_irradiance_matrix(ground_truth_visibility,selected_points)

    gt_mesh = ground_truth_visibility.mesh.mesh
    final_irradiances = 80*irradiance_matrix.transpose()@solutions[solutions>0]
    areas = gt_mesh.area_faces
    colors = gt_mesh.visual.face_colors
    label = colors[:,0] >= 255
    disinfected = final_irradiances > 280
    total_ht_area = np.sum(areas[label])
    total_area = np.sum(areas)
    disinfected_ht_area = np.sum(areas[label][disinfected[label]])
    disinfected_area = np.sum(areas[disinfected])
    ht_area_fraction = total_ht_area/total_area
    return mesh_name,total_area,ht_area_fraction,disinfected_ht_area/total_ht_area,disinfected_area/total_area



sa_roadmap = sorted(glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/surface_agnostic/armbot/armbot_roadmap_330_divs.p'))
# sa_sampling_places = glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/surface_agnostic/armbot/armbot_sampling_places_330_divs.p')


sst_roadmap = sorted(glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/soft_semantic_thresholding/armbot/armbot_roadmap_330_divs.p'))
# sst_sampling_places = glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/soft_sematic_thresholding/armbot/armbot_sampling_places_330_divs.p')


In [ ]:
# result = evaluate_semantic_disinfection_performance(sst_roadmap[0])

In [ ]:
roadmaps = sorted(glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/*/armbot/armbot_roadmap_330_divs.p'))


In [ ]:
mesh_names = []
experiments = []
durations = []
total_areas = []
ht_area_fractions = []
ht_disinfected_fraction = []
total_disinfected_fraction = []
old_mesh = ''
for rm in tqdm(roadmaps):
    experiment = rm.split('/')[-3]
    duration = experiment.split('_')[-2]
    experiments.append(experiment)
    durations.append(duration)
    results = evaluate_semantic_disinfection_performance(rm)
    mesh_names.append(results[0])
    total_areas.append(results[1])
    ht_area_fractions.append(results[2])
    ht_disinfected_fraction.append(results[3])
    total_disinfected_fraction.append(results[4])
results_df = pd.DataFrame({'mesh_name':mesh_names,'experiment':experiments,'duration':durations,'room area':total_areas,
                          'ht_area_fraction':ht_area_fractions,'ht_disinfected_fraction':ht_disinfected_fraction,'total_disinfected_fraction':total_disinfected_fraction})

In [ ]:
results_df.to_csv('./Semantic_results_shorter_time.csv',sep = '|')

In [ ]:
results_df.loc[results_df.mesh_name == 'scene0011_00']

In [ ]:
estimated_visibility = Visibility(mesh_file,res = 512, useShader = True,createWnd = True)
ground_truth_visibility = Visibility(mesh_gt_file,res = 512, useShader = True,createWnd = True)

In [ ]:
mesh_names[-1]

In [ ]:
this_mesh = '/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/scene0011_00.ply'

In [ ]:
a = tm.exchange.load.load(this_mesh)
a.show()

In [ ]:
def get_sampling_places(mesh_file,resolution = 5000,robot_height = 1.5,convex_scale = 0.9,alpha = 2):
    resolution = resolution/1000
    mesh = get_maximum_connected_component_mesh(mesh_file)
    projection = mesh.vertices[mesh.vertices[:,2]<0.3,:2]

    alpha_shape = alphashape(projection,alpha)
    max_area = 0
#     if(len(alpha_shape) > 1):
#         for shape in alpha_shape:
#             if(shape.area>max_area):
#                 max_area = shape.area
#                 final_shape = shape
#         alpha_shape = final_shape
    bounds = alpha_shape.bounds
    x_ = np.arange(bounds[0] - resolution,bounds[2]+resolution,resolution)
    y_ = np.arange(bounds[1] - resolution,bounds[3]+resolution,resolution)
    z_ = np.arange(0,robot_height+resolution,resolution)
    x,y,z = np.meshgrid(x_,y_,z_,indexing = 'ij')
    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    sampling_places = np.zeros(shape = (z.shape[0],3))
    sampling_places[:,0] = x
    sampling_places[:,1] = y
    sampling_places[:,2] = z
    print(sampling_places.shape)
    multipoint = shapely.geometry.MultiPoint(sampling_places[:,:2])
    result = []
    for point in list(multipoint):
        result.append(alpha_shape.contains(point))
    sampling_places = sampling_places[result]
    print("\n\n\n\n this is the shape of sampling places  = {} \n\n\n\n\n".format(sampling_places.shape))
    
    return sampling_places,mesh,alpha_shape,multipoint
a, b, alpha_shape,mp = get_sampling_places(this_mesh,resolution = 250)

In [ ]:
alpha_shape.area

In [ ]:
b.show()

# Plotting Code

In [ ]:
from glob import glob 
import pandas as pd
from matplotlib import pyplot as plt

all_meshes = glob('/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/*.ply')

meshes_series = pd.Series(all_meshes)
estimated_mask = ~meshes_series.str.split('/',expand = True).iloc[:,-1].str.startswith('gt')
# we now get the number of done meshes

# done_files = glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/{}/armbot/*'.format(experiment))
# if(done_files):
#     done_meshes = pd.Series(done_files).str.split('/',expand = True).iloc[:,-4].tolist()
# else:
#     done_meshes = []

estimated_meshes = sorted(meshes_series[estimated_mask].tolist())

In [ ]:
a = ['this is bullshit {}','this is super bullshit {}']
b = a[0].format(12)
a

In [ ]:
import pandas as pd
df = pd.read_csv('./Semantic_results_shorter_time.csv',sep = '|')

In [ ]:
tmp = df.experiment.str.split('_',expand = True)
names = tmp.loc[:,0] + '_' + tmp.loc[:,1]
names[names.str.startswith('hard')] += '_' + tmp.loc[names.str.startswith('hard'),2]
df['experiment_names'] = names

df2 = df[df.duration < 100]
max_disinf = df2.groupby(['mesh_name']).total_disinfected_fraction.max()

In [ ]:
for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'HT uplift vs experiment duration {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = (tmp2.ht_disinfected_fraction/max_disinfection-tmp2.total_disinfected_fraction/max_disinfection),x = tmp2.duration,label = name)
    plt.legend()
    plt.xlabel('Experiment Duration (min)')
    plt.ylabel('ht_fraction - total_fraction \nasymptotic (n.d.)')
    plt.ylim([-0.1,0.1])
    plt.savefig('./media/semantic_results/{}.pdf'.format(title))
    plt.show()

In [ ]:
tmp